In [23]:
import torch
import torch.nn.functional as F
from main import GPT2, GPTConfig
from transformers import GPT2Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2(GPTConfig())
model.load_state_dict(torch.load('epochs_25141.pth'))
model.to('cuda')
enc = GPT2Tokenizer.from_pretrained('gpt2')
enc

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [26]:
model.eval()
num_return_sequences = 4
max_length = 32
tokens = enc.encode("Hello I am pragateesh")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
while xgen.size(1) < max_length:
    with torch.no_grad():
        # with torch.autocast(device_type=device, dtype=torch.bfloat16):
        logits, _ = model(xgen)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng)
        xcol = torch.gather(topk_indices, -1, ix)
        xgen = torch.cat((xgen, xcol), dim=1)
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"\nsample {i}: \n{decoded}\n")


sample 0: 
Hello I am pragateesh with my own idea, as it appears in the next three sentences

There is a question where many, even many a people


sample 1: 
Hello I am pragateesh, the only person in the world with a free spirit. I'm sorry I am so strong, but my powers are not yours


sample 2: 
Hello I am pragateesh, but I don't want to do my own training at this level, but I would like to spend my life studying in the


sample 3: 
Hello I am pragateesh; it's an honest way of living. If you can do something very simple with something trivial to your liking, then I can

